In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/sentiment140/training.1600000.processed.noemoticon.csv
/kaggle/input/sample/scraped_tweets.csv


In [2]:
import nltk
# nltk.download("movie_reviews")
import numpy as np
from nltk.corpus import movie_reviews 
import pandas as pd
from nltk.corpus import twitter_samples 
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
stop = stopwords.words('english')
import string
import re
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegressionCV
from nltk.tokenize import sent_tokenize, word_tokenize
from sklearn.metrics import accuracy_score
from sklearn.metrics import plot_confusion_matrix

from sklearn.ensemble import StackingClassifier, VotingClassifier

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import LinearSVC

from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification
from sklearn.metrics import average_precision_score
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC
import pickle

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## Helper Methods for Cleaning Data 

In [3]:
# Preprocessing
# code here
def remove_all_punctuations(data):
    all_punctuations =  list(string.punctuation)
    all_punctuations.append("``")
    lst = []
    for review in data:
        tokenized = word_tokenize(review)
        considered_word = [word for word in tokenized if not word in all_punctuations]
        considered_word = " ".join(considered_word)
        lst.append(considered_word)
    return lst

def remove_hashtags(data):
        data= [review.replace("#", '"') for review in data]
        return data

# code here
def remove_stop_words(data):
    stop_words = set(stop)
    lst = []
    for review in data:
        tokenized = word_tokenize(review)
        considered_word = [word for word in tokenized if not word in stop_words]
        considered_word = " ".join(considered_word)
        lst.append(considered_word)
    return lst

def remove_links(data):
    ans = []
    for d in data:
         ans.append(re.sub(r"http\S+", "", d))
    return ans

def clean_html(data):
    ans = []
    html = re.compile('<.*?>')
    for text in data:
        
        ans.append( html.sub(r'', text) )
    return ans

def remove_non_ascii(data):
    ans =[]
    for s in data:
        ans.append("".join(i for i in s if ord(i)<128))
    return ans

def clean_links(data):
    ans = []
    txt = re.compile('http(s)?://\w+(\.\w+){1,}(/\w+)*')
    for text in data:
        ans.append(txt.sub(r'', text))
    return ans

def clean_links_a1(data):
    ans = []
    txt = re.compile(r'//t.co/[a-zA-Z0-9_]*')
    for text in data:
        ans.append(txt.sub(r'',text))
    return ans


## Reading Data & Missing Values 

In [4]:
DATASET_COLUMNS  = ["sentiment", "ids", "date", "flag", "user", "text"]
DATASET_ENCODING = "ISO-8859-1"
dataset = pd.read_csv('/kaggle/input/sentiment140/training.1600000.processed.noemoticon.csv', encoding=DATASET_ENCODING , names=DATASET_COLUMNS)
df = dataset
df.head()

,sentiment,ids,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [5]:
X = df['text']
y = df['sentiment']

X_dev, X_test, y_dev, y_test = train_test_split(X,y,test_size=0.2)

print(X_dev.shape)
print(X_test.shape)
print(y_dev.shape)
print(y_test.shape)

(1280000,)
(320000,)
(1280000,)
(320000,)


## Cleaning TEXT for NLP 

In [6]:

X_test = remove_hashtags(X_test)
print("Removed HashTags")
X_test = clean_links(X_test)
print("Removed Links")
#//t.co/DIID0JGmaX
X_test = clean_links_a1(X_test)
print("Removed Links Part 2")
X_test = remove_stop_words(X_test)
print("Removed Stopwords")
X_test = remove_all_punctuations(X_test)
print("Removed Punctuations")

X_dev = remove_hashtags(X_dev)
print("Removed HashTags")
X_dev = clean_links(X_dev)
print("Removed Links")
#//t.co/DIID0JGmaX
X_dev = clean_links_a1(X_dev)
print("Removed Links Part 2")
X_dev = remove_stop_words(X_dev)
print("Removed Stopwords")
X_dev = remove_all_punctuations(X_dev)
print("Removed Punctuations")

Removed HashTags
Removed Links
Removed Links Part 2
Removed Stopwords
Removed Punctuations
Removed HashTags
Removed Links
Removed Links Part 2
Removed Stopwords
Removed Punctuations


## Lemmatizing 

In [7]:
#code here
#Ref : From lecture notest
def stemSentence(sentence):
         porter = PorterStemmer()
         token_words = word_tokenize(sentence)
         stem_sentence = [porter.stem(word) for word in token_words]
         return " ".join(stem_sentence.copy())
print("Starting lemmatization ")

X_dev = [stemSentence(review) for review in X_dev ]
print("Xdev lemmatization done")
X_test = [stemSentence(review) for review in X_test ]
print("Xtestlemmatization done")

print("Lemmatization Completed")


Starting lemmatization 
Xdev lemmatization done
Xtestlemmatization done
Lemmatization Completed


## TFIDF Vectorization

In [8]:
#code here
tfidf_vector = TfidfVectorizer()
X_dev_tfidf = tfidf_vector.fit_transform(X_dev)
print("Done vectorization for X Dev")
X_dev_tfidf = tfidf_vector.transform(X_dev)
print("Done vectorization for X test")

Done vectorization for X Dev
Done vectorization for X test


## Vanilla Vectorization

In [9]:
vector = CountVectorizer()
X_dev_text = vector.fit_transform(X_dev)
print("Done vectorization for X Dev")
X_test_text = vector.transform(X_test)
print("Done vectorization for y dev")

Done vectorization for X Dev
Done vectorization for y dev


## Fit Vanilla Model

In [10]:
model =  LinearSVC(random_state=0, tol=1e-5, verbose=1)
model.fit(X_dev_text, y_dev)
# save the model to disk

[LibLinear]....................................................................................................
optimization finished, #iter = 1000

Using -s 2 may be faster (also see FAQ)

Objective value = -558627.693738
nSV = 1003272


/opt/conda/lib/python3.7/site-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,


LinearSVC(random_state=0, tol=1e-05, verbose=1)

## Fit Tfidf Model

In [11]:
lr_tfid =  LinearSVC(random_state=0, tol=1e-5, verbose=1)
lr_tfid.fit(X_dev_tfidf,y_dev)
print("Logistic Regression Completed for TFIDF Phase")

[LibLinear]......*
optimization finished, #iter = 67
Objective value = -612697.972553
nSV = 1037116
Logistic Regression Completed for TFIDF Phase


## Saving Model Files

In [12]:
filename = 'finalized_model_SVM.sav'
pickle.dump(model, open(filename, 'wb'))

filename_tfidf = 'finalized_model_tfidf_SVM.sav'
pickle.dump(model, open(filename_tfidf, 'wb'))

pickle.dump(vector, open("vectorizer_SVM.pickle", "wb"))

pickle.dump(tfidf_vector, open("vectorizer_tfidf_SVM.pickle", "wb"))

In [13]:
## Using Pretrained Model
directory_path = "/kaggle/input/sample"
!zip -r file.zip directory_path

	zip warning: name not matched: directory_path

zip error: Nothing to do! (try: zip -r file.zip . -i directory_path)


### End of file